In [54]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files

import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from geopy.geocoders import Nominatim

from tqdm.auto import tqdm

df_orig = pd.read_csv('./data/data.csv')

In [55]:
geolocator = Nominatim(user_agent="my_geocoder")

def convert(cell_val):
    """
    Purpose: 
    """
    pattern = r"\d+(,\d+)?(\.\d+)?"
    num_str = re.search(pattern, cell_val)

    if (num_str):
        # comment: 
        return float(num_str.group().replace(',' , ''))
    else:
        return None
        # comment: 
    # end if
# end def

def addr_to_coords_lat(addr):
    """
    Purpose: 
    """
    geolocator = Nominatim(user_agent="my_geocoder", scheme='http', domain='localhost:8080', timeout=10)

    # Геокодирование адреса
    location = geolocator.geocode(addr)

    if (location):
        # comment:
        return location.latitude
    else:
        # comment: 
        return None
    # end if
# end def

def addr_to_coords_lon(addr):
    """
    Purpose: 
    """
    geolocator = Nominatim(user_agent="my_geocoder", scheme='http', domain='localhost:8080', timeout=10)

    # Геокодирование адреса
    location = geolocator.geocode(addr)

    if (location):
        # comment:
        return location.longitude
    else:
        # comment: 
        return None
    # end if
# end def

In [ ]:
df = df_orig.copy()

In [ ]:
df.drop(['zipcode', 'mls-id', 'MlsId'], axis=1, inplace=True)

In [ ]:
df['street'] = df['street'].astype(str).str.strip()
df['state'] = df['state'].astype(str).str.strip()

df['full_addr'] = df['street'] +', '+ df['state']

In [ ]:
df[df['state']=='DC']

In [ ]:
df.info

In [59]:
temp_df = df[df['state']=='DC']

In [60]:
temp_df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,beds,state,stories,PrivatePool,target,full_addr
25,for sale,NaN,single-family home,1755 18th St NW,7 Baths,"{'atAGlanceFacts': [{'factValue': '1915', 'fac...",yes,Washington,"[{'rating': ['10/10', '3/10'], 'data': {'Dista...","5,796 sqft",8 Beds,DC,3.0,NaN,"$3,749,000","1755 18th St NW, DC"
138,for sale,NaN,condo,2660 Connecticut Ave NW #3,4 Baths,"{'atAGlanceFacts': [{'factValue': '1928', 'fac...",yes,Washington,"[{'rating': ['9/10', '10/10'], 'data': {'Dista...","4,532 sqft",3 Beds,DC,1.0,NaN,"$8,095,000","2660 Connecticut Ave NW #3, DC"
225,Active,NaN,Federal,3087 Ordway St NW,NaN,"{'atAGlanceFacts': [{'factValue': '1989', 'fac...",1,Washington,"[{'rating': ['10', '8', '9'], 'data': {'Distan...","4,900",NaN,DC,NaN,NaN,"2,879,000","3087 Ordway St NW, DC"
330,Active,NaN,Condo/Townhome/Row Home/Co-Op,805 Jefferson St NW,3.5,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",NaN,Washington,"[{'rating': ['7', '4', '3', '6', '4', '6', 'NR...",2042,4,DC,3+,NaN,"$799,950","805 Jefferson St NW, DC"
531,for sale,NaN,single-family home,601 Pennsylvania Ave NW #212,NaN,"{'atAGlanceFacts': [{'factValue': '1991', 'fac...",NaN,Washington,"[{'rating': ['8/10', '3/10'], 'data': {'Distan...",801 sqft,NaN,DC,3.0,NaN,"$449,000","601 Pennsylvania Ave NW #212, DC"


In [ ]:
df_orig.head(10)

In [61]:
temp_df.sample(5)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,beds,state,stories,PrivatePool,target,full_addr
201268,Active,NaN,Condo/Townhome/Row Home/Co-Op,448 Buchanan St NW,3.5,"{'atAGlanceFacts': [{'factValue': '1927', 'fac...",NaN,Washington,"[{'rating': ['7', '4', '3', '8', '6', '8', 'NR...",2040,3,DC,3+,NaN,"$779,500","448 Buchanan St NW, DC"
126066,Active,NaN,Condo/Townhome/Row Home/Co-Op,1201 I St NE,1.5,"{'atAGlanceFacts': [{'factValue': '1900', 'fac...",NaN,Washington,"[{'rating': ['5', '6', '4', '3', 'NR', 'NR', '...",2069,3,DC,2,NaN,"$1,100,000","1201 I St NE, DC"
250357,Active,NaN,Condo/Townhome/Row Home/Co-Op,7 18th St SE Apt 103,1,"{'atAGlanceFacts': [{'factValue': '1923', 'fac...",NaN,Washington,"[{'rating': ['4', '3', '4', 'NR', 'NR', 'NR', ...",400,1,DC,1,NaN,"$219,900","7 18th St SE Apt 103, DC"
105480,For sale,NaN,Condo,1920 3rd St NE APT 1,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",NaN,Washington,"[{'rating': ['2/10', 'NA', '3/10'], 'data': {'...",Total interior livable area: 862 sqft,3 bd,DC,3.0,NaN,"$485,000","1920 3rd St NE APT 1, DC"
135415,Active,NaN,Contemporary,88 V St SW #406,NaN,"{'atAGlanceFacts': [{'factValue': '2020', 'fac...",NaN,Washington,"[{'rating': ['NR', 'NR', 'NR', '4', '4'], 'dat...",751,Bath,DC,NaN,NaN,"614,010","88 V St SW #406, DC"


In [62]:
tqdm.pandas()

temp_df['lat_coords'] = temp_df['full_addr'].progress_apply(addr_to_coords_lat)

  0%|          | 0/4674 [00:00<?, ?it/s]

100%|██████████| 4674/4674 [08:14<00:00,  9.46it/s]


In [63]:
temp_df

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,beds,state,stories,PrivatePool,target,full_addr,lat_coords
25,for sale,NaN,single-family home,1755 18th St NW,7 Baths,"{'atAGlanceFacts': [{'factValue': '1915', 'fac...",yes,Washington,"[{'rating': ['10/10', '3/10'], 'data': {'Dista...","5,796 sqft",8 Beds,DC,3.0,NaN,"$3,749,000","1755 18th St NW, DC",38.913919
138,for sale,NaN,condo,2660 Connecticut Ave NW #3,4 Baths,"{'atAGlanceFacts': [{'factValue': '1928', 'fac...",yes,Washington,"[{'rating': ['9/10', '10/10'], 'data': {'Dista...","4,532 sqft",3 Beds,DC,1.0,NaN,"$8,095,000","2660 Connecticut Ave NW #3, DC",38.933605
225,Active,NaN,Federal,3087 Ordway St NW,NaN,"{'atAGlanceFacts': [{'factValue': '1989', 'fac...",1,Washington,"[{'rating': ['10', '8', '9'], 'data': {'Distan...","4,900",NaN,DC,NaN,NaN,"2,879,000","3087 Ordway St NW, DC",38.936498
330,Active,NaN,Condo/Townhome/Row Home/Co-Op,805 Jefferson St NW,3.5,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",NaN,Washington,"[{'rating': ['7', '4', '3', '6', '4', '6', 'NR...",2042,4,DC,3+,NaN,"$799,950","805 Jefferson St NW, DC",38.955451
531,for sale,NaN,single-family home,601 Pennsylvania Ave NW #212,NaN,"{'atAGlanceFacts': [{'factValue': '1991', 'fac...",NaN,Washington,"[{'rating': ['8/10', '3/10'], 'data': {'Distan...",801 sqft,NaN,DC,3.0,NaN,"$449,000","601 Pennsylvania Ave NW #212, DC",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376922,Active,NaN,Tudor,235 Emerson St NW #104,"2,000","{'atAGlanceFacts': [{'factValue': '1929', 'fac...",NaN,Washington,"[{'rating': ['7', 'NR', '8', 'NR', '3'], 'data...",700,Baths,DC,NaN,NaN,"237,500","235 Emerson St NW #104, DC",NaN
376970,for sale,NaN,condo,3476 Summit Ct NE,NaN,"{'atAGlanceFacts': [{'factValue': '1978', 'fac...",yes,Washington,"[{'rating': ['3/10', '3/10', '9/10'], 'data': ...","1,396 sqft",2 Beds,DC,1.0,NaN,"$299,995","3476 Summit Ct NE, DC",38.928052
376977,Active,NaN,Transitional,3113 Albemarle St NW,NaN,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Washington,"[{'rating': ['9', '9', '9'], 'data': {'Distanc...","6,955",NaN,DC,NaN,NaN,"3,495,000","3113 Albemarle St NW, DC",38.947912
377111,Active,NaN,Bungalow,2801 Channing St NE,NaN,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",NaN,Washington,"[{'rating': ['3', '2', '5', '3', '3'], 'data':...","1,732",NaN,DC,NaN,NaN,"637,500","2801 Channing St NE, DC",38.922205


In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df.iloc[1:2]

In [ ]:
# Создание объекта геокодера
geolocator = Nominatim(user_agent="my_geocoder", scheme='http', domain='localhost:8080', timeout=1)

# Геокодирование адреса
location = geolocator.geocode("1755 18th St NW, DC")

# Получение координат
print((location.latitude, location.longitude))


In [ ]:
df.sample(5)

In [ ]:
df['target'] = df['target'].astype(str)
df['target'] = df['target'].map(convert)

In [ ]:
df['private pool'].value_counts()

In [ ]:
df['private pool'].value_counts()

In [ ]:
df['private pool'].fillna(0, inplace=True)

In [ ]:
df.loc[df['private pool']=='Yes', 'private pool'] = 1

In [ ]:
display(df.iloc[16:17])

In [ ]:
display(df_orig.iloc[81106:81107])
display(df.iloc[81106:81107])

In [ ]:
with open('temp.csv', 'w') as f:
    # Comment: 
    df['baths'].value_counts().to_csv(f)
# end open file


In [ ]:
df[df['beds']=='Baths']

In [ ]:
df[df['beds']=='Baths']

In [ ]:
print(df.shape)
print(df_orig.shape)